In [1]:
from brain_observatory_qc.data_access import from_lims
from pathlib import Path
import pandas as pd
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache as bpc
cache = bpc.from_lims()
table = cache.get_ophys_experiment_table(passed_only=False)

c:\users\jinho.kim\github\allensdk\allensdk\brain_observatory\behavior\behavior_project_cache\tables\util\prior_exposure_processing.py:165: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  counts = df.groupby(['mouse_id'])['to'].apply(cumsum)


In [2]:

csv_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pipeline_validation\code_ocean_uploading_temp')
csv_fn = 'ribo_table.csv'
ribo_table = pd.read_csv(csv_dir / csv_fn)

In [4]:
ribo_table['opids'] = ribo_table.session.apply(lambda x: from_lims.get_ophys_experiment_ids_for_ophys_session_id(x).ophys_experiment_id.values)
ribo_table

,expression_strategy,mouse_id,session,opids
0,RiboL1-GCaMP8m(icv),667825,1265619832,"[1265760434, 1265760436]"
1,RiboL1-GCaMP8m(icv),667825,1262862975,"[1263579241, 1263579243]"
2,RiboL1-GCaMP8m(icv),667826,1267183073,"[1267294087, 1267294085]"
3,RiboL1-GCaMP8m(icv),667826,1263821545,"[1263922955, 1263922953]"
4,RiboL1-GCaMP8m(icv),667827,1265369630,"[1265531034, 1265531036]"
5,RiboL1-GCaMP8m(icv),667827,1264602251,"[1264716248, 1264716250]"
6,RiboL1-GCaMP8m(RO),644542,1219401841,"[1219594260, 1219594261, 1219594258, 121959425..."
7,RiboL1-GCaMP8m(RO),644542,1220570937,"[1220831479, 1220831481, 1220831488, 122083148..."
8,RiboL1-GCaMP8m(RO),644542,1226777215,"[1226979376, 1226979371, 1226979374, 122697936..."
9,RiboL1-GCaMP8m(RO),644542,1223477772,"[1223613440, 1223613438, 1223613435, 122361343..."


In [21]:
def get_depth_region_from_db(oeid):
    from visual_behavior import database as db
    query = '''
    SELECT oe.id, st.acronym, oe.calculated_depth, imd.depth
    FROM ophys_experiments oe
    JOIN structures st ON st.id = oe.targeted_structure_id
    JOIN imaging_depths imd ON imd.id = oe.imaging_depth_id
    WHERE oe.id = {}
    '''.format(oeid)
    lims_results = db.lims_query(query)
    return lims_results.depth.values[0], lims_results.acronym.values[0]

In [30]:
ribo_table.columns

Index(['expression_strategy', 'mouse_id', 'session', 'opids'], dtype='object')

In [28]:
opids = ribo_table.opids.explode().values
new_table = pd.DataFrame(opids, columns=['opid'])
new_table['depth'] = 0
new_table['region'] = ''
new_table['session_id'] = 0
new_table['mouse_id'] = 0
new_table['expression_strategy'] = ''
for opid in opids:
    depth, region = get_depth_region_from_db(opid)
    new_table.loc[new_table.opid == opid, 'depth'] = depth
    new_table.loc[new_table.opid == opid, 'region'] = region
    osid = from_lims.get_ophys_session_id_for_ophys_experiment_id(opid)
    new_table.loc[new_table.opid == opid, 'session_id'] = int(osid)
    mouse_id = ribo_table.query('session == @osid').mouse_id.values[0]
    new_table.loc[new_table.opid == opid, 'mouse_id'] = int(mouse_id)
    expression_strategy = ribo_table.query('session == @osid').expression_strategy.values[0]
    new_table.loc[new_table.opid == opid, 'expression_strategy'] = expression_strategy
new_table.set_index('opid', inplace=True)


C:\Users\jinho.kim\Github\visual_behavior_analysis\visual_behavior\database.py:729: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\jinho.kim\Github\visual_behavior_analysis\visual_behavior\database.py:729: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\jinho.kim\Github\visual_behavior_analysis\visual_behavior\database.py:729: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\jinho.kim\Github\visual_behavior_analysi

In [29]:
save_fn = 'ribo_table_with_depth_region.csv'
new_table.to_csv(csv_dir / save_fn)